In [ ]:
import json
import logging
import os
from datetime import datetime, timezone

from dotenv import load_dotenv
from openai import OpenAI
from pymongo import MongoClient, UpdateOne
from pymongo.errors import BulkWriteError
from rich.logging import RichHandler

from bson.objectid import ObjectId
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)

load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
Database_Name = os.getenv("DATABASE")
edps_claims_collection = os.getenv("EDPS_CLAIMS_COLLECTION")
pharmacy_claims_collection = os.getenv("PHARMACY_CLAIMS_COLLECTION")
eligibility_collection = os.getenv("ELIGIBILITY_COLLECTION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
mbi_crosswalk_collection = os.getenv("MBI_CROSSWALK_COLLECTION")
suspects_collection = os.getenv("SUSPECTS_COLLECTION")
staging_suspects_collection = os.getenv("STAGING_SUSPECTS_COLLECTION")

LLM_MODEL = "gpt-4o-mini"
llm_client = OpenAI(api_key=OPENAI_API_KEY)

MBI_CROSSWALK_MAP = {}


In [ ]:
def get_mbi_crosswalk_map():
    """Fetch MemberID -> MBI mapping from Mongodb and return as dict."""
    try:
        with MongoClient(MONGODB_URI) as client:
            db = client[Database_Name]
            projection = {"_id": 0, "created_dt": 0}
            cursor = db[mbi_crosswalk_collection].find({}, projection)

            crosswalk_map = {
                doc.get("MemberID"): doc.get("MBI")
                for doc in cursor
                if doc.get("MemberID") is not None and doc.get("MBI") is not None
            }

            return crosswalk_map

    except Exception as e:
        logging.error(f"Failed to load MBI crosswalk map: {e}")
        return {}


In [ ]:
logging.info("Loading MBI crosswalk map into memory...")
MBI_CROSSWALK_MAP = get_mbi_crosswalk_map()
logging.info(f"Loaded {len(MBI_CROSSWALK_MAP)} crosswalk records.")

In [ ]:
def load_members_with_claims_from_docs(eligibility_docs):
    """Given eligibility documents, fetch medical & pharmacy claims using MBI if available."""
    
    global MBI_CROSSWALK_MAP

    try:
        client = MongoClient(MONGODB_URI)
        db = client[Database_Name]
    except Exception as e:
        logging.error(f"Error connecting to MongoDB: {e}")
        return []

    members = []

    for el in eligibility_docs:
        member_id = el["memberId"]

        # MBI fallback logic
        mbi_lookup_id = MBI_CROSSWALK_MAP.get(member_id, member_id)

        # ---- Medical Claims (EDPS) ----
        medical_claims_cursor = db[edps_claims_collection].find(
            {"Member.Subscriber_ID": mbi_lookup_id},
            {
                "_id": 0,
                "Diagnosis.Diag_Codes": 1,
                "ServiceLine.LXServiceNo": 1,
                "ServiceLine.BilledCPT_Code": 1,
                "ServiceLine.BilledCPTDesc": 1,
                "ServiceLine.Line_SvcDate": 1,
                "Claim.ClaimID": 1,
                "Claim.POS": 1,
                "Type_of_Bill": 1,
                "Provider.BillProv_NPI": 1,
                "Provider.BillProv_LastName": 1,
                "Member.Subscriber_ID": 1,
                "Member.Subscriber_DOB": 1,
                "Member.Subscriber_Gender": 1,
            },
        )
        medical_claims = list(medical_claims_cursor)

        # ---- Pharmacy Claims ----
        pharmacy_claims_cursor = db[pharmacy_claims_collection].find(
            {"Member ID": member_id},
            {
                "_id": 0,
                "Member ID": 1,
                "NDC": 1,
                "Product Label Name": 1,
                "Fill Date": 1,
                "Days Supply": 1,
                "Metric Quantity": 1,
                "Prescriber ID": 1,
                "Prescriber Name": 1,
                "Total Billed": 1,
            },
        )

        pharmacy_claims = [
            {
                "memberId": pc.get("Member ID"),
                "ndc": str(pc.get("NDC")),
                "drugName": pc.get("Product Label Name"),
                "fillDate": (
                    pc.get("Fill Date").strftime("%Y-%m-%d")
                    if pc.get("Fill Date")
                    else None
                ),
                "daysSupply": pc.get("Days Supply"),
                "quantityDispensed": pc.get("Metric Quantity"),
                "prescriberNPI": pc.get("Prescriber ID"),
                "prescriberName": pc.get("Prescriber Name"),
                "totalBilled": pc.get("Total Billed"),
            }
            for pc in pharmacy_claims_cursor
        ]

        members.append(
            {
                "memberId": member_id,
                "eligibility": el,
                "medicalClaims": medical_claims,
                "pharmacyClaims": pharmacy_claims,
            }
        )

    return members

In [ ]:
def undot_keys(document):
    """Recursively convert keys containing dots into nested dictionaries."""
    if isinstance(document, list):
        return [undot_keys(item) for item in document]

    if not isinstance(document, dict):
        return document

    new_doc = {}
    for key, value in document.items():
        if "." in key:
            parts = key.split(".")
            current = new_doc
            for p in parts[:-1]:
                current = current.setdefault(p, {})
            current[parts[-1]] = undot_keys(value)
        else:
            new_doc[key] = undot_keys(value)

    return new_doc


def load_members_claims_to_database(batch_size=100):
    try:
        client = MongoClient(MONGODB_URI)
        db = client[Database_Name]
        logging.info("Connected to MongoDB")
    except Exception as e:
        logging.error(f"Error connecting to MongoDB: {e}")
        return

    suspects_coll = db[staging_suspects_collection]

    cursor = db[eligibility_collection].find(
        {}, {"_id": 0, "createdAt": 0, "updatedAt": 0},
        no_cursor_timeout=True
    ).batch_size(batch_size)

    batch = []
    total_processed = 0

    for el in cursor:
        batch.append(el)

        if len(batch) >= batch_size:
            logging.info(f"Processing batch of {len(batch)}...")

            members = load_members_with_claims_from_docs(batch)
            logging.info(f"Loaded claims for {len(members)} members")

            ops = []
            now = datetime.now(timezone.utc)

            for m in members:
                # ---- SANITIZE ALL DOCUMENTS (fixes BulkWriteError) ----
                m["eligibility"] = undot_keys(m["eligibility"])
                m["medicalClaims"] = [undot_keys(c) for c in m["medicalClaims"]]
                # pharmacyClaims typically clean, but apply too
                m["pharmacyClaims"] = [undot_keys(c) for c in m["pharmacyClaims"]]

                # timestamps
                m["updatedAt"] = now
                m.setdefault("createdAt", now)

                ops.append(
                    UpdateOne(
                        {"memberId": m["memberId"]},
                        {"$set": m, "$setOnInsert": {"createdAt": now}},
                        upsert=True
                    )
                )

            if ops:
                try:
                    suspects_coll.bulk_write(ops, ordered=False)
                except BulkWriteError as bwe:
                    logging.error(f"Bulk write error: {bwe.details}")
                    raise

            total_processed += len(batch)
            logging.info(f"Total processed so far: {total_processed}")

            batch = []

    # ---- Final Batch ----
    if batch:
        logging.info(f"Processing final batch of {len(batch)}...")

        members = load_members_with_claims_from_docs(batch)
        logging.info(f"Loaded claims for {len(members)} members")

        ops = []
        now = datetime.now(timezone.utc)

        for m in members:
            m["eligibility"] = undot_keys(m["eligibility"])
            m["medicalClaims"] = [undot_keys(c) for c in m["medicalClaims"]]
            m["pharmacyClaims"] = [undot_keys(c) for c in m["pharmacyClaims"]]

            m["updatedAt"] = now
            m.setdefault("createdAt", now)

            ops.append(
                UpdateOne(
                    {"memberId": m["memberId"]},
                    {"$set": m, "$setOnInsert": {"createdAt": now}},
                    upsert=True
                )
            )

        if ops:
            try:
                suspects_coll.bulk_write(ops, ordered=False)
            except BulkWriteError as bwe:
                logging.error(f"Bulk write error: {bwe.details}")
                raise

        total_processed += len(batch)

    logging.info(f"Processing complete. Total: {total_processed}")

In [ ]:
# load members and claims into the database collection staging_suspects_collection
load_members_claims_to_database(batch_size=100)